In [ ]:
from bbg_data.core.session import session
from bbg_data.api.options import OptionChainFetcher
from typing import Any
import pandas as pd
import datetime as dt

In [11]:
AS_OF_DATE = dt.date(2025,1,26)

In [12]:
with session() as bbg_session:
    fetcher = OptionChainFetcher(bbg_session)
    nvidia_option_tickers = fetcher.get_option_chain("NVDA US Equity", as_of_date=AS_OF_DATE)
    print(f"Found {len(nvidia_option_tickers)} option tickers for NVDA US Equity")
    nvidia_options_all_data = fetcher.get_option_market_data(
        tickers=nvidia_option_tickers, as_of_date=AS_OF_DATE
    )

Found 4390 option tickers for NVDA US Equity


In [13]:
nvidia_options_all_data

Response(data=[OptionMarketData(contract=OptionContract(ticker='BBG01LLDRYM2 Equity', strike=30.0, expiry=datetime.date(2025, 2, 21), option_type=<OptionType.CALL: 'CALL'>, underlying='NVDA US'), as_of_date=datetime.date(2025, 1, 26), settlement_price=None, last_price=None, bid=None, ask=None, volume=0.0, open_interest=0, implied_vol=None, delta=None, gamma=None, theta=None, vega=None, raw_data={'OPT_STRIKE_PX': 30.0, 'OPT_PUT_CALL': 'Call', 'OPT_EXPIRE_DT': datetime.date(2025, 2, 21), 'OPT_UNDL_TICKER': 'NVDA US', 'VOLUME': 0.0, 'OPEN_INT': 0}), OptionMarketData(contract=OptionContract(ticker='BBG01LLDRYN1 Equity', strike=30.0, expiry=datetime.date(2025, 2, 21), option_type=<OptionType.PUT: 'PUT'>, underlying='NVDA US'), as_of_date=datetime.date(2025, 1, 26), settlement_price=None, last_price=None, bid=None, ask=None, volume=0.0, open_interest=0, implied_vol=None, delta=None, gamma=None, theta=None, vega=None, raw_data={'OPT_STRIKE_PX': 30.0, 'OPT_PUT_CALL': 'Put', 'OPT_EXPIRE_DT': da

In [14]:
records: list[dict[str, Any]] = []
for data in nvidia_options_all_data.data:
    record = {
        "security": data.contract.ticker,
        "strike": data.contract.strike,
        "expiry": data.contract.expiry,
        "option_type": data.contract.option_type.value,
        "settlement_price": data.settlement_price,
        "last_price": data.last_price,
        "bid": data.bid,
        "ask": data.ask,
        "mid_price": data.mid_price,
        "volume": data.volume,
        "open_interest": data.open_interest,
        "implied_vol": data.implied_vol,
        "delta": data.delta,
        "gamma": data.gamma,
        "theta": data.theta,
        "vega": data.vega,
    }
    records.append(record)

nvidia_options_df = pd.DataFrame(records)

In [15]:
nvidia_options_df.to_csv(f"nvda_options_data_{AS_OF_DATE.isoformat()}.csv", index=False)

In [17]:
TENOR = "1M"

with session() as bbg_session:
    fetcher = OptionChainFetcher(bbg_session)
    nvidia_atm_df = fetcher.get_50delta_vol_timeseries(
    underlying="NVDA US Equity",
    start_date=dt.date(2025, 1, 1),
    end_date=dt.date(2025, 3, 31),
    tenor=TENOR
    )

In [19]:
nvidia_atm_df.data.to_csv("nvda_atm_{TENOR}.csv", index=False)

In [20]:
nvidia_atm_df.errors

[]